<a href="https://colab.research.google.com/github/TomoharuKurosu/TomoharuKurosu/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['Capital Share'] = 1 - data['labsh']  # Capital share
data['TFP Share'] = data['labsh']
data['y_pc'] = np.log(data['rgdpna'] / (data['emp']**data['labsh']))  # GDP per worker


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['Growth Rate'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['Capital Deepening'] = data['Growth Rate']*data['Capital Share']
data['TFP Growth'] = data['Growth Rate']*data['TFP Share']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'Growth Rate': 'mean',
                                            'TFP Growth': 'mean',
                                           'Capital Deepening': 'mean',
                                           'TFP Share': 'mean',
                                       'Capital Share': "mean"}
                                       )

# Print output 
display(summary)

,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
countrycode,,,,,
CAN,1.706878,1.092808,0.614070,0.653810,0.346190
DEU,1.205051,0.743555,0.461496,0.629270,0.370730
FRA,1.070213,0.657406,0.412807,0.618806,0.381194
GBR,0.829919,0.478775,0.351144,0.587007,0.412993
ITA,0.300761,0.143510,0.157251,0.515165,0.484835
JPN,1.388959,0.783808,0.605151,0.569399,0.430601
USA,2.108495,1.276218,0.832277,0.606740,0.393260
